- Tous les notebooks sont accesibles aussi [ici](https://github.com/thalitadru/CoursNNDL)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
plt.style.use('fivethirtyeight')

In [ ]:
def plot_y(name):
    if name == 'train':
        y_ = y_train.reshape(-1)
        X_ = X_train
    elif name== 'test':
        y_ = y_test.reshape(-1)
        X_ =X_test
    else:
        print("Il faut taper train ou test")
        return 
    y_pred_ = y_pred(X_).reshape(-1)
    order = np.argsort(y_pred_)
    plt.plot(y_[order], 'o ', label=u'réel ')
    plt.plot(y_pred_[order], 'P ', label=u'prédiction ')
    plt.legend()
    plt.xlabel(u'échantillons')
    plt.ylabel('valeur de y')

# Pratique 1 Descente du gradient avec NumPy
Ici nous allons implementer la descente du gradient pour un modèle de regression lineaire. On travailera en laguage python, avec des biliothèques qui permenttent de faire du calcul scientifique. Ici on commencera avec numpy.

In [ ]:
import numpy as np

## Bases: vecteurs et matrices sur NumPy

Numpy va nous permettre de créer des matrices et vecterus, et de faire des operations mathématiques assez facilement. On peut créer des tableaux de la taille qu'on veut. Un vecteur d'une seulle dimension se fait ainsi:

In [ ]:
np.array([1,2,3])

Ce vecteur n'est ni ligne ni colonne, car il a une seule dimension.

On peut creer un vrai vecteur-ligne de la façon suivante:

In [ ]:
np.array([[1,2,3]])

Et un vecteur colonne de façon similaire

In [ ]:
np.array([[1],[2],[3]])

Une façon plus courte de l'écrire est de l'écrire en ligne et ensuite demander son transposé.

In [ ]:
np.array([[1,2,3]]).T

Avec la même fonction on peut creer une matrice

In [ ]:
np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

On peut sauvegarder les matrices et vecteurs dans une variable, pour les utiliser plus tard:

In [ ]:
M = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
v = np.array([[2,2,2,2]]).T

In [ ]:
M

In [ ]:
v

### Vérifier la taille avec `shape`

On peut verifier la taille d'un vecteur ou matrice en regardant leur variable `shape`.

In [ ]:
M.shape

Cela nous dit que M a 4 lignes et 3 colonnes. L'ordre est toujours ligne puis colonne.

In [ ]:
v.shape

Cela nous dit que v a 4 lignes et une colonne (donc un vecteur colone)

### Opérations le long d'une ligne ou colonne

On peut réaliser des opérations qui combinent les élements des lignes ou des colonnes d'une matrice: la somme `sum`, la moyenne `mean`, la variance `std`, le `max` ou le `min`.

Prennons par exemple une matrice de 1s de taille 3x5:


In [ ]:
A = np.ones((3,5))
A

Si on veut sommer au long de chaque colonne, on peut faire le suivant:

In [ ]:
A.sum(axis=0)

In [ ]:
np.sum(A, axis=0)

Si on veut le faire au long de chaque ligne, on utilise `axis=1`

In [ ]:
A.sum(axis=1)

In [ ]:
np.sum(A, axis=1)

Le même se fait pour les autres opperations. Par example la moyenne:

In [ ]:
A.mean(axis=0)

In [ ]:
A.mean(axis=1)

### Produit de matrices

On peut aussi faire un produit de matrices ou de vecteurs. Par exemple, je veux calculer le produit de $v^T M$, je peux le faire avec la fonction `np.dot`. Cela doit nous retourner un vecteur-ligne de taille 1x3.

In [ ]:
np.dot(v.T, M)

On peut aussi faire $M^T v$. Cela doit nous retourner un vecteur-colonne de taille 3x1.

In [ ]:
np.dot(M.T, v)

### Multiplication élement à élement

Un autre opération utile est la multiplication élément à élement. Pour l'éxemplifier, on va créer d'abord une matrice pleine de 1s:

In [ ]:
A = np.ones((3,3)) #on met ici la taille de la matrice que l'on souhaite, ici 3x3, representé (3,3)
A

Disons que l'on veuille multiplier la premiere ligne par 1, la deuxiéme par 2, et ainsi de suite. On peut creer un vecteur avec ces coeficients de la façon suivante:

In [ ]:
x=np.array([[1,2,3]]).T
x

On peut aussi utiliser la mèthode `np.arange` que nous fournit un vecteur avec une suite de valeurs demandée:

In [ ]:
x = np.arange(1,3+1)  # le vecteur sera rempli de chiffres de start à stop-1, pour cela il faut dire de 1 a 3+1
x

Par contre ici on reçoit un vector de dimension 1. 

In [ ]:
x.shape

Pour le metre en mode ligne ou colone il faut adapter sa taille (`shape`). Dans sa grande dimension, on met un -1, qui dit a numpy de reprendre la dimension di vector, pour la dimension petite, on met un 1. Comme ça on peut creer un vector ligne ou colonne:

In [ ]:
x = x.reshape((1,-1))
x

In [ ]:
x = x.reshape((-1,1))
x

Maintenant on peut réaliser la multiplication de la façon suivante:

In [ ]:
A*x

Remarquez que chaque ligne a été multiplié par un des termes du vector. Ça arrive parce que x est en forme colonne. Pour le faire pour chaque ligne on peut transposer x:

In [ ]:
A*x.T

### Opérations avec scalaires

Opérations avec des scalaires sont faites directement. 

In [ ]:
A

In [ ]:
A+1

In [ ]:
3*A

In [ ]:
x

In [ ]:
x-1

### Autres opérations élement à élement

Quand on fait des operations entre des matrices ou vecteurs de même taile, cela se fait élement à element:

In [ ]:
x+x

In [ ]:
x*x

Avec ces informations, on peut déjà partir à implementer la régression lineaire.

## Les données

On va travailler ici avec un jeu de données synthetique simples. Le code ci dessus vous prepare ce jeux de données. Il contient plus de détails que ce qu'on aura le temps de discuter, vous n'avez pas besoin de le comprendre. 

In [ ]:
# Make data
rng=np.random.RandomState(0)
m = 800
std=1
X1 = rng.rand(m) 
X2 = rng.rand(m) 
#X1, X2 = np.meshgrid(X1, X2)
Y = 3*X1 - 5*X2 + 8 
Y += std*rng.rand(*Y.shape)

X, y = np.stack([X1.reshape(-1), X2.reshape(-1)],axis=1), Y.reshape(-1)

Pour vérifier que notre modèle generalise bien pour des cas inconnus, il est important de separer une portion des données qui ne sera pas utilisé dans l'entraînement, qu'on appelle ensemble de test ou validation.  Ceci est fait ci-desous:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

y_train, y_test = y_train.reshape([-1,1]), y_test.reshape([-1,1])

Avec les données generés, je vous afficherais ici le jeu de données qu'on utilisera pour l'entrainement. 

In [ ]:
plt.subplot(1,3,1)
plt.title("x2 contre x1 - y en couleurs")
plt.scatter(*X_train.T, c=y_train.squeeze(), cmap=cm.coolwarm)
plt.colorbar()
plt.xlabel("x1")
plt.ylabel("x2")
plt.subplot(1,3,2)
plt.title("y contre x1")
plt.scatter(X_train[:,0], y_train.squeeze(), c=y_train.squeeze(), cmap=cm.coolwarm)
plt.xlabel('x1')
plt.ylabel('y')
plt.subplot(1,3,3)
plt.title("y contre x2")
plt.scatter(X_train[:,1], y_train.squeeze(), c=y_train.squeeze(), cmap=cm.coolwarm)
plt.xlabel('x2')
plt.ylabel('y')
plt.gcf().set_size_inches(16,4)

Notre objectif ici est de predire la valeur y (couleur) a partir des deux coordonées x1 et x2.

### Exercice 
Regardez la dimension de X_train, X_test, y_train, y_test

In [ ]:
# votre code ici
X_train.shape, y_train.shape, X_test.shape, y_test.shape

On a donc `X_train` et `y_train` pour apprendre notre regression lineaire. Passons à l'écriture du modèle.

## Le modèle

Comme discuté, une regression linéaire a la forme suivante
$$ y= Xw + b$$

On peut l'écrire de la façon suivante. D'abord, selon la taille des données, on crée le vecteur w et le biais b.

In [ ]:
M = X_train.shape[0]  # nombre d'échantillons
D = X_train.shape[1]  # nombre de features

rng=np.random.RandomState(0)
w = rng.rand(D,1)
b = np.array(0.0)

assert w.shape == (2,1)
assert b.shape == () 

w,b

Ensuite on peut écrire la formule de prediction. Ici on va utiliser une fonction courte qui s'écrit dans une seule ligne (`lambda` function).

In [ ]:
y_pred = lambda x: (np.dot(x,w) + b)

Pour l'utiliser il faut l'appeler avec les données `x` en argument. On utilisera les données d'entrainement `X_train`:

In [ ]:
y_pred(X_train)

In [ ]:
y_pred(X_train).shape

## La fonction de coût

On doit aussi évaluer les predictions faites. On doit donc écrire la fonction de coût:
$$ J(w_j) = \frac{1}{2M}\sum_i (y_I -\hat{y}_i)^2$$

On peut écrire cete fonction de la façon suivante (en profitant des opérations faites élément par élement pour calculer $y_i - \hat{y}$ por tout $i$):

In [ ]:
J = lambda x, y : (1/2)*np.mean((y_pred(x)-y)**2)

Comme pour `y_pred`, on doit passer les arguments de la fonction quand on l'appele. Ici, `X_train` et `y_train`.

In [ ]:
J(X_train, y_train)

### Exercice : Le gradient de J

Pour le gradient de J ça sera três similaire. Pour rappel:
 
$$\nabla_w J = \frac{1}{M} (X^T(y -\hat{y}))$$
et
$$ \frac{\partial J}{\partial b} = \frac{1}{M}\sum_i  (y_i -\hat{y}_i)$$

à vous de l'écrire ci desous:

In [ ]:
# gradJW = lambda x, y: 0 # écrivez l'expression en termes de x et y
# gradJb = lambda x, y: 0 # écrivez l'expression en termes de x et y
gradJW = lambda x, y : (1/M) * np.dot(x.T,(y_pred(x)-y))
gradJb = lambda x, y : np.mean((y_pred(x)-y))

In [ ]:
dw = gradJW(X_train, y_train)
db = gradJb(X_train, y_train)

In [ ]:
# quelques testes pour verifier le bon format de vos gradients
assert dw.shape == w.shape
assert db.shape == b.shape

## La boucle de descente du gradient

Pour ajuster $w$ et $b$ via méthode du gradient, il nous faut itérer en boucle sur le calcul du gradient et l'ajustement de w. Pour cela on s'utilisera de l'instruction `for i in range(max_iterations)` que répet les instructions qui sont dans son intérieur `max_iterations` fois.

On peut voir que les predictions ne sont pas tres bones pour l'instant.

In [ ]:
plot_y('train')

### Exercice
Completez le code avec les pas de descente du gradient:

$$w_j = w_j - \lambda \frac{\partial J}{\partial w_j}$$
$$b = b - \lambda \frac{\partial J}{\partial b}$$

In [ ]:
# lancez ce code si vous avez besoin de reinitializer w et b
rng=np.random.RandomState(0)
w = rng.rand(D,1)
b = np.array(0.0)

assert w.shape == (2,1)
assert b.shape == () 

w, w.shape, b, b.shape

In [ ]:
max_iterations = 100000 

l = 5e-5  # lambda

tol = 1e-7

J_now = J(X_train, y_train)

cost_curve = [J_now]

for i in range(max_iterations):

    # apliquer le pas du gradient a w et b
    # VOTRE CODE ICI
    # w = 
    # b = 
    w = w - l * gradJW(X_train, y_train)
    b = b - l *gradJb(X_train,y_train)
    

    # verifier la valeur de la fonction de cout J
    J_old = J_now

    J_now = J(X_train, y_train)
    
    cost_curve += [J_now]
    # verifier conditions d'arrêt selon la valeur de J
    if np.isnan(J_now):
        break
    if i > 3:
        if np.abs(J_now)< tol and np.abs(J_now-J_old) < tol and np.abs(J_old-cost_curve[-3]) < tol:
            print("arret par convergence @ tol=", tol)
            break
    

    if not i % (max_iterations//10): 
        print("i", i," cout", J_now)
if i >= max_iterations: print("arret par max_iterations ", max_iterations)

Ici vous pouvez regarder la valeur de la fonction de cout au long des itérations.

In [ ]:
plt.plot(cost_curve)
plt.ylabel("cost")
plt.xlabel("iterations")

In [ ]:
plot_y('train')

## Evaluer le modèle
Regardons maintenant la capacité de notre modèle a faire des prédictions sur des données pas vues pendant l'entraînement.

In [ ]:
print("valeur de la fonction de cout sur l'ensemble de test:", J(X_test, y_test))
plot_y('test')

#### Exercice : essayez de changer le nombre d'iterations ou changer lambda pour voir si on arrive a un résultat different

### Comparer avec la solution exacte
La solution optimale à une régression lineaire peut être calculé de manière analytique comme fait ci dessous:
$$X_1 = (X,\textbf{1})$$
$$ (w|b) = (X_1^T X_1)^{-1}X_1^T\hat{y}$$


In [ ]:
Xones = np.concatenate([X_train, np.ones([M,1])], axis=1)
wb = np.dot(np.dot(np.linalg.pinv(np.dot(Xones.T,Xones)), Xones.T), (y_train))
w = wb[:-1]
b = wb[-1]
w,w.shape, b, b.shape

On peut voir que la descente de gradient nous rapproche de la solution, mais ne trouve pas forcement l'optimum dans un nombre d'iterations limitées.

Regardez ici la valeur de la fonction de cout avec cette solution optimale et aussi comment les valeurs predites sont plus proches des vraies valeures.

In [ ]:
J(X_train, y_train)

In [ ]:
plot_y('train')

In [ ]:
J(X_test, y_test)

In [ ]:
plot_y('test')

## Corrigés
Voici les bons morceaux de code à remplir pour que la manip fonctionne.

### Gradients de J
``` python
gradJW = lambda x, y : (1/M) * np.dot(x.T,(y_pred(x)-y))
gradJb = lambda x, y : np.mean((y_pred(x)-y))
```
### Pas du gradient
``` python
for i in range(max_iterations):
    
    # apliquer le pas du gradient a w et b
    # VOTRE CODE ICI
    w = w - l * gradJw(X_train, y_train)
    b = b - l *gradJb(X_train,y_train)
    
    ...
```